# `FStream` Offline Training

This notebook performs offline training of the **flow stream** on the **DAVIS 2016** dataset.

MaskRNN's' binary segmentation net is a 2-stream convnet (`astream` and `fstream`):

![](img/maskrnn.png)

Section "3.3 Binary Segmentation" of the MaskRNN paper and "Figure 2" are inconsistent when it comes to describing the inputs of the two-stream network. In this implementation, we chose the input of the flow stream `fstream` to be the concatenation of the magnitude of the flow field from <sub>t-1</sub> to I<sub>t</sub> and I<sub>t</sub> to frame I<sub>t+1</sub> and the warped prediction of the previous frame's segmentation mask b<sub>t-1</sub>, denoted as φ<sub>t-1,t</sub>(b<sub>t-1</sub>). The warping function φ<sub>t-1,t</sub>(.) transforms the input based on the optical flow fields from frame I<sub>t-1</sub> to frame I<sub>t</sub>. The `FStream` network takes 3-channel inputs (||φ<sub>t-1,t</sub>||, ||φ<sub>t,t+1</sub>||, φ<sub>t-1,t</sub>(b<sub>t-1</sub>)):

The offline training of the `AStream` is done using a **3-channel input** VGG16 network pre-trained on ImageNet:

![](img/osvos_parent.png)


To monitor training, run:
```
tensorboard --logdir E:\repos\tf-video-seg\tfvos\models\fstream_parent
http://localhost:6006
```

In [ ]:
"""
fstream_offline_training.ipynb

FStream offline trainer

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Based on:
  - https://github.com/scaelles/OSVOS-TensorFlow/blob/master/osvos_parent_demo.py
    Written by Sergi Caelles (scaelles@vision.ee.ethz.ch)
    This file is part of the OSVOS paper presented in:
      Sergi Caelles, Kevis-Kokitsi Maninis, Jordi Pont-Tuset, Laura Leal-Taixe, Daniel Cremers, Luc Van Gool
      One-Shot Video Object Segmentation
      CVPR 2017
    Unknown code license
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
from PIL import Image
import numpy as np
import tensorflow as tf
slim = tf.contrib.slim
import matplotlib.pyplot as plt

In [ ]:
# Import model files
import model
import datasets

## Configuration

In [ ]:
# Offline training parameters
imagenet_ckpt = 'models/vgg_16_3chan.ckpt' # copy of checkpoint in http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz
segnet_stream = 'fstream'
ckpt_name= segnet_stream + '_parent'
logs_path = os.path.join('models', ckpt_name)
gpu_id = 0
iter_mean_grad = 10
max_training_iters_1 = 15000
max_training_iters_2 = 30000
max_training_iters_3 = 50000
save_step = 5000
test_image = None
display_step = 100
ini_learning_rate = 1e-8
boundaries = [10000, 15000, 25000, 30000, 40000]
values = [ini_learning_rate, ini_learning_rate * 0.1, ini_learning_rate, ini_learning_rate * 0.1, ini_learning_rate,
          ini_learning_rate * 0.1]

## Dataset load

In [ ]:
# Load DAVIS 2016 dataset
options = datasets._DEFAULT_DAVIS16_OPTIONS
# Set the following to True if you have a lot of RAM
options['data_aug'] = False
# Set the following to wherever you have downloaded the DAVIS 2016 dataset
dataset_root = 'E:/datasets/davis2016/' if sys.platform.startswith("win") else '/media/EDrive/datasets/davis2016/'
train_file = dataset_root + 'ImageSets/480p/train.txt'
dataset = datasets.davis16(train_file, None, dataset_root, options)

In [ ]:
# Display dataset configuration
dataset.print_config()

## Offline Training

In [ ]:
# Train the network with strong side outputs supervision
with tf.Graph().as_default():
    with tf.device('/gpu:' + str(gpu_id)):
        global_step = tf.Variable(0, name='global_step', trainable=False)
        learning_rate = tf.train.piecewise_constant(global_step, boundaries, values)
        model.train_parent(dataset, imagenet_ckpt, 1, learning_rate, logs_path, max_training_iters_1, save_step,
                           display_step, global_step, segnet_stream, iter_mean_grad=iter_mean_grad, test_image_path=test_image,
                           ckpt_name=ckpt_name)

In [ ]:
# Train the network with weak side outputs supervision
with tf.Graph().as_default():
    with tf.device('/gpu:' + str(gpu_id)):
        global_step = tf.Variable(max_training_iters_1, name='global_step', trainable=False)
        learning_rate = tf.train.piecewise_constant(global_step, boundaries, values)
        model.train_parent(dataset, imagenet_ckpt, 2, learning_rate, logs_path, max_training_iters_2, save_step,
                           display_step, global_step, segnet_stream, iter_mean_grad=iter_mean_grad, resume_training=True,
                           test_image_path=test_image, ckpt_name=ckpt_name)

In [ ]:
# Train the network without side outputs supervision
with tf.Graph().as_default():
    with tf.device('/gpu:' + str(gpu_id)):
        global_step = tf.Variable(max_training_iters_2, name='global_step', trainable=False)
        learning_rate = tf.train.piecewise_constant(global_step, boundaries, values)
        model.train_parent(dataset, imagenet_ckpt, 3, learning_rate, logs_path, max_training_iters_3, save_step,
                           display_step, global_step, segnet_stream, iter_mean_grad=iter_mean_grad, resume_training=True,
                           test_image_path=test_image, ckpt_name=ckpt_name)